Mhviraf's [kernel](https://www.kaggle.com/mhviraf/synthetic-data-for-next-instant-gratification) shows the data might be generated by using sklearn's `make_classification`

Using QDA I got a few train rows with high oof score & label 0, or low oof score & label 1.

So I change these data's target to see if it works.

Most part of this notebook is taken from Roman's [kernel](https://www.kaggle.com/nroman/i-m-overfitting-and-i-know-it) & Chris's [kernel](https://www.kaggle.com/cdeotte/pseudo-labeling-qda-0-969)

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm import tqdm_notebook
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.svm import NuSVC
import multiprocessing
from scipy.optimize import minimize  
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [10]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic'] == i]
    test2 = test[test['wheezy-copper-turtle-magic'] == i]
    idx1 = train2.index
    idx2 = test2.index

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    # feature selection
    data2 = VarianceThreshold(threshold = 2).fit_transform(data[cols])

    train3 = pd.DataFrame(data2[:train2.shape[0]], index = idx1)
    train3['target'] = train2['target']
    test3 = pd.DataFrame(data2[train2.shape[0]:], index = idx2)
    
    # Kfold
    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train3, train3['target']):
        # clf
        #clf = QuadraticDiscriminantAnalysis(0.5)
        clf = NuSVC(probability = True, kernel = 'poly')
        X_train = train3.iloc[train_index, :].drop(["target"], axis = 1)
        X_test = train3.iloc[test_index, :].drop(["target"], axis = 1)
        y_train = train3.iloc[train_index, :]['target']
        y_test = train3.iloc[test_index, :]['target']
        clf.fit(X_train, y_train)
        #log = LogisticRegression()
        #clf = QuadraticDiscriminantAnalysis(0.5)
        #log.fit(X_train, y_train)
        # output
        train_prob = clf.predict_proba(X_train)[:,1]
        test_prob = clf.predict_proba(X_test)[:,1]
        oof[idx1[test_index]] = test_prob
        
        # bagging
        preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits
        print("Chunk {0} Fold {1}".format(i, roc_auc_score(y_test, test_prob)))

auc = roc_auc_score(train['target'], oof)
print(f'AUC: {auc:.5}')

Chunk 0 Fold 0.8816666666666667
Chunk 0 Fold 0.8466666666666667
Chunk 0 Fold 0.89
Chunk 0 Fold 0.87
Chunk 0 Fold 0.885
Chunk 0 Fold 0.9483333333333333
Chunk 0 Fold 0.8833333333333333
Chunk 0 Fold 0.9045138888888888
Chunk 0 Fold 0.8559027777777778
Chunk 0 Fold 0.9131944444444444
Chunk 0 Fold 0.9148550724637681
Chunk 1 Fold 0.8333333333333333
Chunk 1 Fold 0.9311594202898551
Chunk 1 Fold 0.9221014492753623
Chunk 1 Fold 0.9202898550724639
Chunk 1 Fold 0.9337121212121211
Chunk 1 Fold 0.9185606060606061
Chunk 1 Fold 0.9507575757575758
Chunk 1 Fold 0.8958333333333334
Chunk 1 Fold 0.8011363636363638
Chunk 1 Fold 0.962121212121212
Chunk 1 Fold 0.9375
Chunk 2 Fold 0.9376181474480151
Chunk 2 Fold 0.8695652173913044
Chunk 2 Fold 0.891304347826087
Chunk 2 Fold 0.8794466403162056
Chunk 2 Fold 0.8557312252964426
Chunk 2 Fold 0.7747035573122529
Chunk 2 Fold 0.8695652173913043
Chunk 2 Fold 0.9525691699604744
Chunk 2 Fold 0.8972332015810276
Chunk 2 Fold 0.8715415019762847
Chunk 2 Fold 0.8925619834710743

KeyboardInterrupt: 

In [16]:
500*512

256000

In [4]:
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    data2 = VarianceThreshold(threshold=2).fit_transform(data[cols])

    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):
        log = LogisticRegression()
        #clf = QuadraticDiscriminantAnalysis(0.5)
        log.fit(train3[train_index,:],train2.loc[train_index]['target'])

        #clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        
        
        
        oof[idx1[test_index]] = log.predict_proba(train3[test_index,:])[:,1]
        preds[idx2] += log.predict_proba(test3)[:,1] / skf.n_splits

auc = roc_auc_score(train['target'], oof)
print(f'AUC: {auc:.5}')


AUC: 0.80697


In [5]:
train2 = train[train['wheezy-copper-turtle-magic']==i]
test2 = test[test['wheezy-copper-turtle-magic']==i]

In [6]:
test['target'] = preds
test.loc[test['target'] > 0.99, 'target'] = 1
test.loc[test['target'] < 0.01, 'target'] = 0

In [7]:
usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]
new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
new_train.loc[oof > 0.99, 'target'] = 1
new_train.loc[oof < 0.01, 'target'] = 0

In [8]:
oof2 = np.zeros(len(train))
preds2 = np.zeros(len(test))
for i in tqdm_notebook(range(512)):

    train2 = new_train[new_train['wheezy-copper-turtle-magic']==i]
    test2 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train[train['wheezy-copper-turtle-magic']==i].index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    data2 = StandardScaler().fit_transform(VarianceThreshold(threshold=2).fit_transform(data[cols]))
    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):
        oof_test_index = [t for t in test_index if t < len(idx1)]
        
        log = LogisticRegression()
        #clf = QuadraticDiscriminantAnalysis(0.5)
        log.fit(train3[train_index,:],train2.loc[train_index]['target'])
        if len(oof_test_index) > 0:
            oof2[idx1[oof_test_index]] = log.predict_proba(train3[oof_test_index,:])[:,1]
        preds2[idx2] += log.predict_proba(test3)[:,1] / skf.n_splits
        
auc = roc_auc_score(train['target'], oof2)
print(f'AUC: {auc:.5}')


AUC: 0.80901


In [7]:
sub = pd.read_csv('sample_submission.csv')
sub['target'] = preds2
sub.to_csv('submission.csv',index=False)

It brings ~0.0002 LB boost, be careful using it still.